In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, average_precision_score
from sklearn.preprocessing import label_binarize

In [2]:
df = pd.read_csv("combined_features.csv")

In [3]:
df = df.drop(["Unnamed: 0"],axis=1)
df

,0,1,2,3,4,5,6,7,8,9,...,Corr2,Diss_sim2,Homogen2,Contrast2,Energy3,Corr3,Diss_sim3,Contrast3,Energy4,Target
0,0.192964,-0.019705,0.026618,0.351818,0.298722,-0.107279,-0.015158,0.003892,0.524406,0.039238,...,0.919516,7.661608,0.127554,98.646275,0.015894,0.829719,11.224156,208.034439,0.107351,0
1,0.157366,-0.003816,-0.032858,0.196809,0.330078,0.383103,0.176619,0.198241,0.298677,0.024531,...,0.977489,4.184100,0.220152,29.967895,0.021490,0.947064,6.401126,70.011755,0.107569,0
2,0.161836,-0.049620,-0.011000,0.348439,0.330918,0.024357,0.014045,-0.043150,0.681322,0.188252,...,0.958037,4.263552,0.213115,30.323549,0.023645,0.897296,6.670947,74.221170,0.124114,0
3,0.111617,-0.046071,-0.010727,0.289515,0.345447,-0.014513,-0.073970,-0.052690,0.628684,0.210376,...,0.925150,5.723648,0.166492,54.709969,0.020782,0.828209,8.687137,125.530404,0.123403,0
4,0.064389,-0.069336,-0.089412,0.319757,0.169375,-0.006689,-0.036932,-0.051993,0.562059,0.139916,...,0.978757,3.995765,0.230973,28.129354,0.023903,0.951120,6.041322,64.345599,0.113598,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4184,0.296233,-0.173819,-0.132807,0.344676,0.304321,0.397681,0.205759,0.005951,0.303051,-0.060433,...,0.999487,0.607368,0.749175,1.201333,0.137047,0.998722,0.995182,2.996365,0.203269,5
4185,0.396976,-0.081235,-0.149095,0.266616,0.361952,0.590082,0.108408,0.333073,0.347843,-0.034620,...,0.991617,2.392191,0.403972,14.914763,0.045681,0.980559,3.579858,34.599094,0.129531,5
4186,0.304681,-0.134328,-0.017254,0.295696,0.609226,0.225363,0.309966,0.270593,0.506838,-0.128632,...,0.998685,1.155074,0.648774,4.162545,0.132761,0.996455,1.920197,11.243822,0.181156,5
4187,0.297888,-0.202946,-0.122310,0.371556,0.635890,0.387773,0.210122,0.073176,0.410118,-0.056148,...,0.999119,0.628868,0.756240,1.758523,0.132519,0.997725,1.036403,4.543673,0.201567,5


In [4]:
df["Target"].unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [5]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['Target'])
y = df['Target']
# X = your feature matrix (excluding target column)
# Step 1: Scale the data (very important for PCA!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 2: Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of variance (OR set a fixed number like 50)
X_pca = pca.fit_transform(X_scaled)

print("Original shape:", X.shape)
print("Reduced shape:", X_pca.shape)

Original shape: (4189, 271)
Reduced shape: (4189, 17)


In [6]:


# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Create and train the SVM model
model = SVC(kernel='rbf', C=1.0, gamma='scale',probability=True)  # You can change kernel to 'linear', 'poly', etc.
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

In [7]:
y_probs = model.predict_proba(X_test)

# 2. Convert y_test to binary format for average_precision_score
y_test_bin = label_binarize(y_test, classes=sorted(list(set(y))))

# 3. Compute mean Average Precision
mAP = average_precision_score(y_test_bin, y_probs, average="macro")
print("\n✅ mean Average Precision (mAP):", mAP)


✅ mean Average Precision (mAP): 0.9887228851731219


In [8]:
y_pred

array([2, 4, 4, 0, 2, 0, 0, 5, 1, 2, 5, 5, 2, 1, 2, 2, 0, 1, 0, 4, 2, 5,
       0, 0, 4, 2, 2, 4, 1, 2, 1, 2, 1, 3, 1, 2, 1, 1, 5, 4, 2, 3, 2, 5,
       4, 1, 5, 2, 1, 4, 5, 1, 5, 0, 1, 0, 2, 4, 0, 5, 1, 5, 4, 1, 4, 2,
       5, 5, 1, 1, 5, 2, 0, 4, 4, 4, 0, 2, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       4, 2, 2, 0, 5, 0, 5, 1, 0, 4, 4, 4, 1, 2, 1, 5, 3, 2, 3, 1, 0, 3,
       0, 2, 4, 1, 2, 4, 5, 0, 2, 2, 5, 0, 1, 3, 4, 2, 0, 1, 1, 1, 2, 5,
       1, 1, 4, 2, 3, 1, 2, 1, 0, 4, 5, 3, 3, 2, 0, 5, 2, 4, 3, 2, 1, 1,
       2, 0, 2, 0, 2, 2, 2, 4, 5, 3, 2, 1, 2, 2, 1, 0, 1, 2, 0, 4, 1, 2,
       4, 1, 1, 4, 2, 5, 0, 0, 2, 3, 2, 2, 4, 5, 0, 4, 1, 3, 0, 4, 2, 4,
       4, 5, 2, 4, 1, 5, 1, 0, 1, 2, 3, 2, 4, 0, 2, 0, 5, 3, 2, 1, 2, 3,
       5, 2, 2, 1, 1, 4, 1, 0, 0, 1, 5, 0, 2, 1, 4, 3, 1, 2, 1, 3, 3, 2,
       1, 1, 0, 1, 5, 0, 3, 1, 0, 2, 4, 4, 4, 0, 1, 1, 1, 5, 2, 5, 1, 4,
       1, 1, 3, 0, 1, 4, 2, 2, 0, 0, 0, 1, 4, 3, 5, 2, 3, 1, 0, 4, 1, 4,
       2, 0, 1, 1, 0, 3, 2, 5, 1, 3, 5, 0, 2, 5, 5,

In [9]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9618138424821002

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.96      0.97       159
           1       0.94      0.98      0.96       195
           2       0.99      0.97      0.98       189
           3       0.98      0.97      0.98        64
           4       0.93      0.97      0.95       113
           5       0.95      0.92      0.93       118

    accuracy                           0.96       838
   macro avg       0.96      0.96      0.96       838
weighted avg       0.96      0.96      0.96       838



In [10]:
X_pca


array([[ 8.36382332e+00,  1.14751343e+01,  1.79054123e+01, ...,
        -8.02834374e-01,  1.77912800e+00, -6.90334567e-01],
       [ 9.37323639e+00, -1.54438929e+01,  4.91965882e+00, ...,
        -8.70428190e-01, -4.35706322e-01, -1.11580412e+00],
       [ 1.05674802e+01,  9.84883849e+00,  8.98530078e+00, ...,
        -1.84929988e+00, -2.94073606e-01,  1.78548445e-01],
       ...,
       [-2.05893726e+00, -2.67469359e+00, -5.24829795e+00, ...,
         2.48009891e-01,  8.21424068e-01,  2.31560608e+00],
       [ 1.19061140e+00, -2.36181875e+00, -8.90645148e+00, ...,
         4.17122162e+00,  1.35284440e+00, -9.32431816e-03],
       [-3.83302947e+00, -2.86788692e+00, -6.52477131e+00, ...,
         5.44445063e-01, -1.27384093e+00, -7.23246944e-03]])

In [11]:
import joblib

# Save each component
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(pca, 'pca.pkl')
joblib.dump(model, 'svm_model.pkl')


['svm_model.pkl']

In [12]:
#note to future self, this is how ur supposed to transform ur input as well

scaler = joblib.load("scaler.pkl")
pca = joblib.load("pca.pkl")
model = joblib.load("svm_model.pkl")

# Apply same preprocessing
X_scaled = scaler.transform(X_test)
X_pca = pca.transform(X_scaled)

# Predict
y_pred = model.predict(X_pca)
print("Predictions:", y_pred)

C:\Users\saiet\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: X has 17 features, but StandardScaler is expecting 271 features as input.